In [1]:
%env TORCH_USE_CUDA_DSA=1
%env CUDA_LAUNCH_BLOCKING=1

env: TORCH_USE_CUDA_DSA=1
env: CUDA_LAUNCH_BLOCKING=1


In [1]:
from typing import Tuple
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

import numpy as np

import torch

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Optimizer, Adam
from torch.optim.lr_scheduler import LRScheduler

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
torch.set_default_dtype(torch.float64)

In [4]:
test_df = pd.read_pickle("test_proc.pkl")
test_df

,0,1,2,3,4,5,6,7,8,9,...,1000,element_1,element_2,element_3,element_1_ratio,element_2_ratio,element_3_ratio,temp,pressure,air_ratio
51976,1.429725e-20,3.325318e-20,1.513897e-20,1.119836e-21,1.209848e-21,2.024112e-21,7.172782e-21,4.953834e-21,1.539859e-21,1.285451e-21,...,9.105786e-25,9,12,22,0.136030,0.262144,0.601825,283.0,0.2,0.6
18970,5.940243e-23,7.336601e-23,9.720283e-23,1.447083e-22,2.680262e-22,8.343595e-22,1.010416e-20,6.042984e-21,7.479574e-22,3.643073e-22,...,1.465311e-22,0,21,22,0.188355,0.316809,0.494835,283.0,0.1,0.6
30497,1.193615e-20,6.945339e-21,6.543001e-22,2.529143e-22,1.749403e-22,3.243140e-22,3.550860e-22,1.091960e-22,9.629268e-23,1.882503e-22,...,7.445923e-23,5,16,19,0.444071,0.477905,0.078024,323.0,0.1,0.6
30356,1.624151e-20,5.577240e-21,9.267746e-22,2.168774e-21,5.549700e-21,4.163047e-21,1.670818e-21,8.912111e-22,8.374945e-22,1.444200e-21,...,3.604447e-22,4,21,24,0.509421,0.224513,0.266066,323.0,0.1,0.6
140548,2.003258e-21,1.077312e-21,3.272797e-22,1.452913e-22,9.167639e-23,6.963908e-23,5.832702e-23,5.251748e-23,4.962818e-23,4.851140e-23,...,9.615524e-22,4,13,18,0.227958,0.482563,0.289478,263.0,0.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218031,3.535618e-22,4.722553e-22,4.692500e-22,5.307244e-22,6.048017e-22,6.291295e-22,6.453254e-22,6.485575e-22,7.700791e-22,1.061828e-21,...,8.640137e-25,6,9,-1,0.530767,0.469233,0.000000,323.0,0.8,0.6
57587,9.397569e-22,1.173034e-21,2.279960e-21,8.301202e-22,8.418463e-22,9.303921e-22,1.093095e-21,2.257390e-21,3.849821e-21,3.221007e-21,...,1.020158e-21,14,15,18,0.600135,0.375171,0.024694,303.0,0.2,0.6
254241,7.204798e-22,1.141918e-21,2.021069e-21,3.898292e-21,4.167931e-21,3.322392e-21,2.260052e-21,1.356805e-21,9.869838e-22,1.119172e-21,...,1.174975e-22,10,18,20,0.273032,0.469203,0.257765,293.0,1.0,0.6
30894,4.030195e-22,5.592868e-22,9.234086e-22,2.112966e-21,5.189306e-21,3.964872e-21,1.755474e-21,1.090197e-21,1.088081e-21,1.695364e-21,...,1.690917e-23,8,14,24,0.375711,0.384662,0.239627,323.0,0.1,0.6


In [5]:
class CustomSpectraDataset(Dataset):
    def __init__(self, data: pd.DataFrame, device="cuda:0") -> None:
        self.data = data
        self.elements = self.data["element_1"].unique()
        # self.air_ratios = data.air_ratio.to_numpy(dtype=np.float64)

        self.spectras = torch.log(
            torch.tensor(
                self.data[[str(i) for i in range(1001)]].to_numpy(dtype=np.float64)
            )
        ).to(device)

        self.ratios = torch.tensor(
            self.data[
                ["element_1_ratio", "element_2_ratio", "element_3_ratio"]
            ].to_numpy(dtype=np.float64)
        ).to(device)

        self.element_indices = self.data[
            ["element_1", "element_2", "element_3"]
        ].to_numpy(dtype=np.float64)

        self.elements_distributions = torch.zeros(
            [len(self.data), len(self.elements)], dtype=torch.float64
        ).to(device)

        for idx in range(len(self.data)):
            indices = self.element_indices[idx, :]
            indices = indices[indices != -1]
            self.elements_distributions[idx, indices] = torch.where(self.ratios[idx][
                range(indices.shape[0])
            ] > 0, 1.0, 0.0).double()

        self.elements_distributions = self.elements_distributions[
            ~torch.isnan(self.spectras).any(dim=1)
        ]
        self.spectras = self.spectras[~torch.isnan(self.spectras).any(dim=1)]

    def __len__(self) -> int:
        return len(self.spectras)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        spectra = self.spectras[idx]
        elements_distribution = self.elements_distributions[idx]

        return spectra, elements_distribution

In [6]:
test_dataset = CustomSpectraDataset(test_df)

In [7]:
val_loader = DataLoader(
    test_dataset,
    batch_size=64,
)

In [8]:
class THzResNetBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 3,
        stride: int = 1,
        activation: nn.Module = nn.ReLU(),
        dropout: float = 0.05,
    ) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        assert kernel_size % 2 == 1
        self.layers = nn.Sequential(
            nn.Conv1d(
                in_channels,
                out_channels,
                kernel_size=kernel_size,
                padding=kernel_size // 2,
                stride=stride,
            ),
            nn.Dropout(p=dropout),
            activation,
            nn.Conv1d(
                out_channels,
                out_channels,
                kernel_size=kernel_size,
                padding=kernel_size // 2,
                stride=stride,
            ),
            nn.Dropout(p=dropout),
            activation,
        )
        self.skip_connection = nn.Conv1d(
            in_channels=in_channels, out_channels=out_channels, kernel_size=1
        )
        self.post_processing = nn.Sequential(
            nn.BatchNorm1d(self.out_channels),
            activation,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        unprocessed_result = self.layers(x) + self.skip_connection(x)
        return self.post_processing(unprocessed_result)

In [9]:
class THzBottleNeck(nn.Module):

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 3,
        stride: int = 1,
        activation: nn.Module = nn.ReLU(),
        dropout: float = 0.05,
    ) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        assert kernel_size % 2 == 1
        self.layers = nn.Sequential(
            nn.Conv1d(
                in_channels,
                in_channels // 4,
                kernel_size=1,
                padding=0,
                stride=1,
            ),
            nn.Dropout(p=dropout),
            activation,
            nn.Conv1d(
                in_channels // 4,
                in_channels // 4,
                kernel_size=kernel_size,
                padding=kernel_size // 2,
                stride=stride,
            ),
            nn.Dropout(p=dropout),
            activation,
            nn.Conv1d(
                in_channels // 4,
                out_channels,
                kernel_size=1,
                padding=0,
                stride=1,
            ),
            activation,
            nn.Dropout(p=dropout),
        )
        self.skip_connection = (
            nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=1)
            if in_channels != out_channels
            else lambda x: x
        )
        self.post_processing = nn.Sequential(
            nn.BatchNorm1d(self.out_channels),
            activation,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        unprocessed_result = self.layers(x) + self.skip_connection(x)
        return self.post_processing(unprocessed_result)

In [10]:
class THzCNN(nn.Module):
    def __init__(self, n_elements: int) -> None:
        super().__init__()
        self.n_elements = n_elements
        self.net = nn.Sequential(
            # input_shape: (batch_size, 1, 1001)
            nn.BatchNorm1d(1),
            THzResNetBlock(in_channels=1, out_channels=64, kernel_size=7),
            THzBottleNeck(in_channels=64, out_channels=64, kernel_size=3),
            THzBottleNeck(in_channels=64, out_channels=64, kernel_size=3),
            nn.MaxPool1d(kernel_size=4),
            # input_shape: (batch_size, 64, 250)
            THzResNetBlock(in_channels=64, out_channels=128, kernel_size=3),
            THzBottleNeck(in_channels=128, out_channels=128, kernel_size=3),
            THzBottleNeck(in_channels=128, out_channels=128, kernel_size=3),
            nn.MaxPool1d(kernel_size=5),
            # input_shape: (batch_size, 128, 50)
            THzResNetBlock(in_channels=128, out_channels=256, kernel_size=3),
            THzBottleNeck(in_channels=256, out_channels=256, kernel_size=3),
            THzBottleNeck(in_channels=256, out_channels=256, kernel_size=3),
            nn.MaxPool1d(kernel_size=5),
            # input_shape: (batch_size, 256, 10)
            THzResNetBlock(in_channels=256, out_channels=512, kernel_size=3),
            THzBottleNeck(in_channels=512, out_channels=512, kernel_size=3),
            THzBottleNeck(in_channels=512, out_channels=512, kernel_size=3),
            nn.MaxPool1d(kernel_size=5),
            # input_shape: (batch_size, 512, 2)
            nn.Flatten(),
            # linear head
            nn.Linear(512 * 2, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, self.n_elements),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

In [11]:
net = THzCNN(test_dataset[0][1].shape[0])
net.load_state_dict(torch.load("./cnn-detection.model"))

net.to(device)


print(device)


pred, y_test = np.empty((0, 25)), np.empty((0, 25))
for spectra, target in tqdm(val_loader, desc="testing"):
    net.eval()
    ans = net(spectra[:, None, :])
    ans = nn.Softmax()(ans).cpu().detach().numpy()
    pred = np.append(pred, ans, axis=0)
    y_test = np.append(y_test, target.cpu().numpy(), axis=0)

cuda:0


testing:   0%|          | 0/2048 [00:00<?, ?it/s]C:\Users\whoee\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
testing: 100%|██████████| 2048/2048 [04:09<00:00,  8.20it/s]


In [13]:
# for spectra, target in tqdm(val_loader, desc="testing"):
#     ans = net(spectra)
#     ans = nn.Softmax()(ans)
#     break
# ans[5], target[5]

testing:   0%|          | 0/4095 [00:00<?, ?it/s]


RuntimeError: running_mean should contain 1001 elements not 1

In [12]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    mean_absolute_error,
)

In [14]:
def cross_entropy(predictions, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions.
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1.0 - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets * np.log(predictions + 1e-9)) / N
    return ce

In [17]:
# reg
print(f'MAE: {mean_absolute_error(y_test, pred)}')
print(f"MAE: {mean_absolute_error(y_test, pred, multioutput='raw_values')}")
print(f"Cross Entropy: {cross_entropy(y_test, pred)}")

MAE: 0.019411249832876582
MAE: [0.00640533 0.02327715 0.02247833 0.00866646 0.00921007 0.00726134
 0.01036512 0.00942259 0.02407267 0.01124802 0.00970918 0.0109634
 0.00743144 0.0093933  0.00862645 0.01011655 0.00979261 0.00836744
 0.00904559 0.00907102 0.00802924 0.00961578 0.011302   0.01069435
 0.01134808 0.22877898]
Cross Entropy: 2.105780903335893


In [39]:
# detection
print(f'F1: {f1_score(y_test, np.where(pred > 0.12, 1, 0), average="macro")}')
print(
    f'precision: {precision_score(y_test, np.where(pred > 0.12, 1, 0), average="macro")}'
)
print(f'recall: {recall_score(y_test, np.where(pred > 0.12, 1, 0), average="macro")}')
print(f"accuracy: {accuracy_score(y_test, np.where(pred > 0.12, 1, 0))}")

F1: 0.9402385036137177
precision: 0.9390588682468852
recall: 0.9425612194580725
accuracy: 0.7568512229556988


In [40]:
print(f"precision: {precision_score(y_test, np.where(pred > 0.12, 1, 0), average=None)}")
print(f"recall: {recall_score(y_test, np.where(pred > 0.12, 1, 0), average=None)}")
print(f"F1: {f1_score(y_test, np.where(pred > 0.12, 1, 0), average=None)}")

precision: [1.         0.52313939 0.63120567 1.         0.99993346 0.99740329
 0.89652628 0.99871352 0.46923328 0.99657603 0.99986556 0.9995314
 0.99993337 0.99993265 0.99320996 0.99973378 0.9989235  0.99966808
 0.9997348  1.         0.99886531 1.         0.99152827 0.99953321
 0.98328088]
recall: [0.99834831 0.63031968 0.55364548 0.99292909 0.99675002 0.99727049
 0.96883928 0.97507768 0.57104042 0.97657895 0.98778139 0.99130262
 0.9978722  0.99550758 0.99699298 0.99774161 0.99657672 0.99787953
 0.9963658  0.99660181 0.99073155 0.99118119 0.98556439 0.99409736
 0.98703436]
F1: [0.99917347 0.5717499  0.58988704 0.996452   0.9983392  0.99733688
 0.93128113 0.98675408 0.51515514 0.98647616 0.99378675 0.9954
 0.99890172 0.99771521 0.99509788 0.9987367  0.99774873 0.99877301
 0.99804746 0.99829801 0.9947818  0.99557106 0.98853734 0.99680787
 0.98515404]
